# sw_audio_dsp pipeline designer

In this file you can generate the DSP pipeline of your choice.

Below you will find 4 cells which can be modified and executed to configure, tune and run the desired pipeline.



1. This is the pipeline design cell. Here you must break the DSP pipeline down into threads and use the provided DSP stages to create a pipeline. Running this cell will produce a diagram showing your pipeline. Make sure to capture each stage in your pipeline as a variable, as it will be needed in the next step.
Note that every time the pipeline cell is changed, the app must be regenerated before the tuning stage can work correctly as the stage indices used for communication may have changed.

In [ ]:
# Pipeline design stage

from audio_dsp.design.pipeline import Pipeline
from audio_dsp.stages.signal_chain import Fork, VolumeControl, Switch, Adder, Mixer
from audio_dsp.stages.noise_gate import NoiseGate
from audio_dsp.stages.cascaded_biquads import CascadedBiquads
from audio_dsp.stages.compressor_sidechain import CompressorSidechain
from audio_dsp.stages.envelope_detector import EnvelopeDetectorRMS

# 2 inputs
p = Pipeline(2)

with p.add_thread() as t:
    f_mus = t.stage(Fork, p.i[0])
    mic_vc = t.stage(VolumeControl, p.i[1])
    music_vc = t.stage(VolumeControl, f_mus.forks[0] + f_mus.forks[1])
    f_mic = t.stage(Fork, mic_vc.o)
    env_mic = t.stage(EnvelopeDetectorRMS, f_mic.forks[1])
    peq = t.stage(CascadedBiquads, f_mic.forks[0])
    ng = t.stage(NoiseGate, peq.o)

with p.add_thread() as t:
    f_ng = t.stage(Fork, ng.o, count=4)
    duck0 = t.stage(CompressorSidechain, [music_vc.o[0]] + f_ng.forks[0])
    duck1 = t.stage(CompressorSidechain, [music_vc.o[1]] + f_ng.forks[1])
    f_dck = t.stage(Fork, duck0.o + duck1.o)
    mix_duck = t.stage(Mixer, [f_dck.forks[0][0]] + [f_dck.forks[1][0]])
    add_out = t.stage(Adder, mix_duck.o + f_ng.forks[2])
    sw0 = t.stage(Switch, add_out.o + f_ng.forks[3])
    monitor_vc = t.stage(VolumeControl, [f_dck.forks[0][1]] + [f_dck.forks[1][1]])
    output_vc = t.stage(VolumeControl, sw0.o)
    f_monvc = t.stage(Fork, monitor_vc.o[0])
    env_out = t.stage(EnvelopeDetectorRMS, f_monvc.forks[1])
    
p.set_outputs(f_monvc.forks[0] + [output_vc.o[0]])
p.draw()


2. This is the tuning cell. First time through this can be ignored, but once your pipeline is running on a connected device, this cell can be updated and executed to update each pipeline stage live.

In [ ]:
# Tuning stage

env_mic.make_env_det_rms(0.3, 0.3)
env_out.make_env_det_rms(0.3, 0.3)
peq.make_parametric_eq([["highpass", 50, 0.707]])
ng.make_noise_gate(-30, 0.005, 0.12)
duck0.make_compressor_sidechain(5, -30, 0.01, 0.5)
duck1.make_compressor_sidechain(5, -30, 0.01, 0.5)


3. This is the build and run cell. This stage generates an application which uses your pipeline, and it runs the application on a connected device using xrun. The tuning parameters set in the previous cell are baked in the application.

In [ ]:
# Build and run

from audio_dsp.design.pipeline import generate_dsp_main
# from build_utils import build_and_try_run

generate_dsp_main(p, out_dir="src/audio_dsp/dsp_pipeline")
# build_and_try_run()


4. This is the profile cell. This stage reports several metrics to optimize the performance of the pipeline.